## Configuration for fine-tuning and vLLM inference

In [ ]:
import os
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()

hf_token = os.getenv('HF_TOKEN')
login(token=hf_token) # or login by terminal `huggingface-cli login`

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

###-- Critical Environment Memo --###
# CUDA: 12.9
# Driver version: 575.57.08
# torch version: 2.7.1 + cu128
###-------------------------------###

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# Load the model for fine-tuning
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    token=hf_token,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

# Configure tokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# Clean up model if necessary
del model
torch.cuda.empty_cache()

import gc 
gc.collect()

39

In [5]:
peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=32,  # Rank of decomposed matrices
    lora_alpha=64,  # Scaling factor
    lora_dropout=0.1,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj", "lm_head"
    ],
    bias="none"
)

# Prepare model for training
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
peft_model = get_peft_model(model, peft_config)

## Prepare Dataset

### Financial-Alpaca

In [ ]:
from datasets import load_dataset

# Load Finanace-Alpaca dataset from huggingface
alpaca_data = load_dataset("gbharti/finance-alpaca", split="train")

In [ ]:
import json

def convert_to_jsonl(example):
    system_msg = {"role": "system", "content": "You are a financial expert specialized in SEC filings and market analysis."}
    
    if example["input"]:
        user_msg = f"{example['instruction']}\nInput:\n{example['input']}"
    else:
        user_msg = example["instruction"]
    
    return {
        "messages": [
            system_msg,
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": example["output"]}
        ]
    }

# Process and save as JSONL
with open("financial_alpaca.jsonl", "w") as f:
    for example in alpaca_data:
        converted = convert_to_jsonl(example)
        json.dump(converted, f)
        f.write("\n")


In [17]:
# Validate dataset
!python ~/mulkooo/mistral-finetune/utils/reformat_data.py ~/mulkooo/sj_Trading/content/alpaca/financial_alpaca.jsonl

Skipped 6475th sample


In [ ]:
# Run fine-tuning with alpaca data
# Commands are run under /mistral-finetune:

# python -m torch.distributed.run --nproc_per_node=1 finetune.py financial_training.jsonl \
#     --model_name_or_path mistralai/Mistral-7B-Instruct-v0.3 \
#     --max_steps 1000 \
#     --learning_rate 2e-4 \
#     --seq_len 4096